In [0]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [1]:
!pip install transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 368kB 8.6MB/s 
     |████████████████████████████████| 645kB 75.9MB/s 
     |████████████████████████████████| 1.0MB 73.6MB/s 
     |████████████████████████████████| 860kB 63.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=c018a3df4f9cf3c4bbececa2b54c59a47f2a0e574469d694f0b19f3d36a9f63b
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


100%|██████████| 231508/231508 [00:00<00:00, 900360.07B/s]


In [0]:
len(tokenizer.vocab)

30522

In [0]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [0]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [0]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [0]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [0]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [0]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [0]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [0]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.3MB/s]


In [0]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
print(vars(train_data.examples[6]))

{'text': [3649, 1996, 22617, 1997, 1996, 2143, 1010, 2009, 2003, 9996, 18800, 1012, 2004, 2500, 2031, 4197, 2041, 1010, 1996, 3185, 3065, 10575, 1999, 4238, 4092, 1999, 5640, 1010, 2738, 2084, 4723, 1012, 2008, 2003, 2438, 2000, 4558, 21553, 2005, 3087, 2040, 2038, 1996, 15989, 3716, 1997, 1996, 2181, 2030, 1996, 2406, 1012, 1996, 5957, 2071, 2025, 2022, 2062, 2367, 2013, 1996, 5025, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2060, 25854, 10697, 1024, 1037, 3345, 2003, 3491, 2000, 2022, 4082, 1999, 7041, 1010, 2096, 7041, 2515, 2025, 2031, 7111, 1012, 1996, 5037, 6059, 2003, 4147, 1037, 10768, 2480, 1006, 1996, 2417, 6045, 1007, 1010, 6168, 1996, 10768, 2480, 2001, 7917, 2011, 4977, 2172, 2077, 1996, 2051, 1999, 2029, 1996, 3185, 2003, 2275, 1012, 1996, 5037, 6059, 1005, 1055, 2684, 2003, 2941, 5102, 2004, 2019, 2796, 1010, 1998, 2796, 4556, 2189, 2003, 2652, 1999, 1996, 4281, 1999, 2116, 5019, 1012, 1045, 6814, 1996, 16587, 3214, 2000, 2265, 2019, 12564, 2450, 1010, 1998, 1

In [0]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['whatever', 'the', 'merits', 'of', 'the', 'film', ',', 'it', 'is', 'poorly', 'researched', '.', 'as', 'others', 'have', 'pointed', 'out', ',', 'the', 'movie', 'shows', 'locals', 'in', 'iran', 'speaking', 'in', 'arabic', ',', 'rather', 'than', 'persian', '.', 'that', 'is', 'enough', 'to', 'lose', 'credibility', 'for', 'anyone', 'who', 'has', 'the', 'slightest', 'knowledge', 'of', 'the', 'area', 'or', 'the', 'country', '.', 'the', 'landscape', 'could', 'not', 'be', 'more', 'different', 'from', 'the', 'actual', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'other', 'factual', 'errors', ':', 'a', 'train', 'is', 'shown', 'to', 'be', 'operating', 'in', 'afghanistan', ',', 'while', 'afghanistan', 'does', 'not', 'have', 'railways', '.', 'the', 'turkish', 'ambassador', 'is', 'wearing', 'a', 'fe', '##z', '(', 'the', 'red', 'hat', ')', ',', 'whereas', 'the', 'fe', '##z', 'was', 'banned', 'by', 'turkey', 'much', 'before', 'the', 'time', 'in', 'which', 'the', 'movie', 'is', 'set', '.', 'the', 'tu

In [0]:
LABEL.build_vocab(train_data)

In [0]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fed06473840>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

In [6]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 440473133/440473133 [00:14<00:00, 29503141.50B/s]


In [0]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [0]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [0]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [0]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/Starry-night-in-the-winter-mountains.jpg' mode='r' encoding='UTF-8'>

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    #start_time = time.time()
    print("1")
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    print("hell")
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    print("2")
    #end_time = time.time()
    print("3")
    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print("4")
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './Bert-model.pt')
    
    #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

1
hell
2
3
4
	Train Loss: 0.279 | Train Acc: 88.70%
	 Val. Loss: 0.237 |  Val. Acc: 89.87%
1
hell
2
3
4
	Train Loss: 0.236 | Train Acc: 90.67%
	 Val. Loss: 0.247 |  Val. Acc: 90.12%
1
hell
2
3
4
	Train Loss: 0.212 | Train Acc: 91.62%
	 Val. Loss: 0.231 |  Val. Acc: 91.14%
1
hell
2
3
4
	Train Loss: 0.189 | Train Acc: 92.67%
	 Val. Loss: 0.246 |  Val. Acc: 90.37%
1
hell
2
3
4
	Train Loss: 0.160 | Train Acc: 93.84%
	 Val. Loss: 0.224 |  Val. Acc: 91.04%


We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [11]:
model.load_state_dict(torch.load('./Bert-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<All keys matched successfully>

## Inference

In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
predict_sentiment(model, tokenizer, "This film is great")

0.9411056041717529

In [ ]:
predict_sentiment(model, tokenizer, "This film is so bad!")